In [8]:
import rasterio
import numpy as np
import joblib
import pandas as pd

from rural_beauty.config import data_dir, models_dir 
# from rural_beauty.config import BFN_features_unique, BFN_coefs_unique, BFN_constant_unique




In [9]:
para_outcome = 'diverse'
para_type = 'elastic'
sugar    =  '281024'  # random identifier to have different models with same other paras
model_basename = f"{para_outcome}_{para_type}_{sugar}"
model_folder = models_dir / model_basename


In [ ]:
pd.read_csv()

In [7]:
raster_paths = [v for k,v in features.items()]
coefficients = [coefs[k] for k,v in features.items()]


In [10]:
raster_paths

[PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/dem/DEM_EU_range.tif'),
 PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/dem/neighborhood/DEM_EU_range_zone3.tif'),
 PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/code_seemee.tif'),
 PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/code_heide.tif'),
 PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/protected/WDPA_area.tif'),
 PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/neighborhood/code_natgru_zone1_2.tif'),
 PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/code_wein.tif'),
 PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/neighborhood/code_acker_zone1_2.tif'),
 PosixPath('/h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/neighborhood/code_stoer_zone1_2.tif'),

In [8]:
# New base directory for the adjusted rasters
new_base_dir =  data_dir / 'forprediction/uniqueness'

new_base_dir.mkdir(parents=True, exist_ok=True)

# Create new list of file paths for the adjusted rasters
adjusted_raster_paths = [new_base_dir / raster_path.name for raster_path in raster_paths]


In [9]:
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
import joblib

def subset_normalize_and_align_rasters(raster_paths, output_paths, scaler_path):
    """
    Subset all rasters to the largest area contained within all inputs,
    align them to the same resolution and extent, and normalize the data using a saved scaler.

    Args:
    - raster_paths (list of str): List of file paths to the input rasters.
    - output_paths (list of str): List of output file paths to save the aligned and normalized rasters.
    - scaler_path (str): Path to the .pkl file containing the fitted StandardScaler.
    """
    # Load the scaler
    scaler = joblib.load(scaler_path)

    # Open all rasters and get their bounds
    bounds_list = []
    transforms = []
    crs = None

    for path in raster_paths:
        with rasterio.open(path) as src:
            bounds_list.append(src.bounds)
            transforms.append(src.transform)
            if crs is None:
                crs = src.crs  # Assuming all rasters are in the same CRS, else reproject would be needed.

    # Calculate the largest common intersection (extent)
    intersection_bounds = (
        max([b.left for b in bounds_list]),   # Left bound
        max([b.bottom for b in bounds_list]), # Bottom bound
        min([b.right for b in bounds_list]),  # Right bound
        min([b.top for b in bounds_list])     # Top bound
    )

    ref_resolution = 1000
    pixel_width = ref_resolution
    pixel_height = ref_resolution
    width = int((intersection_bounds[2] - intersection_bounds[0]) / pixel_width)
    height = int((intersection_bounds[3] - intersection_bounds[1]) / pixel_height)

    # Align rasters to this common extent
    for idx, (raster_path, output_path) in enumerate(zip(raster_paths, output_paths)):
        with rasterio.open(raster_path) as src:
            # Calculate the transform for the common extent
            transform, _, _ = calculate_default_transform(
                src.crs, src.crs, width=width, height=height, 
                left=intersection_bounds[0], bottom=intersection_bounds[1], 
                right=intersection_bounds[2], top=intersection_bounds[3])

            # Create a new dataset with the common extent
            kwargs = src.meta.copy()
            kwargs.update({
                'height': height,
                'width': width,
                'transform': transform
            })
            print(f"Aligning and normalizing {raster_path}. Dimensions: {width}, {height}")

            # Read, reproject, and normalize the data, then save
            with rasterio.open(output_path, 'w', **kwargs) as dst:
                for i in range(1, src.count + 1):  # Reproject and normalize all bands
                    # Allocate an array to hold the reprojected data
                    reprojected_data = np.empty((height, width), dtype=src.dtypes[i - 1])
                    
                    reproject(
                        source=rasterio.band(src, i),
                        destination=reprojected_data,
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=src.crs,
                        resampling=Resampling.nearest  # You can change this to another method if needed
                    )

                    # Normalize data (flatten, transform, reshape back)
                    reprojected_data_flat = reprojected_data.flatten().reshape(-1, 1)
                    normalized_data_flat = scaler.transform(reprojected_data_flat)
                    normalized_data = normalized_data_flat.reshape(height, width)

                    # Write the normalized data to the output file
                    dst.write(normalized_data, i)






In [8]:
subset_normalize_and_align_rasters(raster_paths, adjusted_raster_paths, normalize_X_diverse)

Aligning /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/dem/DEM_EU_range.tif. dimensions: 6564, 4058
Aligning /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/dem/neighborhood/DEM_EU_range_zone3.tif. dimensions: 6564, 4058
Aligning /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/code_seemee.tif. dimensions: 6564, 4058
Aligning /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/code_heide.tif. dimensions: 6564, 4058
Aligning /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/protected/WDPA_area.tif. dimensions: 6564, 4058
Aligning /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/neighborhood/code_natgru_zone1_2.tif. dimensions: 6564, 4058
Aligning /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/code_wein.tif. dimensions: 6564, 4058
Aligning /h/u145/hofer/MyDocuments/Granular/beauty/data/cleaned/clc/layer_coverage_EU/neighborhood/code_acker_zone1_2.tif. dimens

In [9]:
import rasterio
import numpy as np

def linear_combination_rasters(raster_paths, coefficients, constant, output_path):
    """
    Create a new raster from a linear combination of input rasters.
    
    Args:
    - raster_paths (list of str): List of file paths to the input rasters.
    - coefficients (list of float): Coefficients for the linear combination, same length as raster_paths.
    - output_path (str): File path to save the output raster.
    """
    # Open the first raster to get metadata
    with rasterio.open(raster_paths[0]) as src:
        meta = src.meta
        data = src.read(1) * coefficients[0]  # Read the first raster and multiply by its coefficient

    # Loop through the remaining rasters and apply the linear combination
    for i, raster_path in enumerate(raster_paths[1:], start=1):
        with rasterio.open(raster_path) as src:
            data += src.read(1) * coefficients[i]  # Add the weighted raster data

    data += constant
    # Update metadata if needed (e.g., datatype)
    meta.update(dtype=rasterio.float64)

    # Save the result as a new raster
    with rasterio.open(output_path, 'w', **meta) as dst:
        dst.write(data.astype(rasterio.float64), 1)




In [11]:
linear_combination_rasters(adjusted_raster_paths, coefficients, constant, data_dir / 'unique.tif')